# <center> <img src="../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Procesamiento de Datos Masivos** </center>
---
### <center> **Primavera 2025** </center>
---
### <center> **Ejemplos de Aprendizaje Automático (Machine Learning): Logistic Regression** </center>

---
**Profesor**: Dr. Pablo Camarillo Ramirez

**Team grandeInformacion members**:
- Miguel Alberto Torres Dueñas
- Juan Pablo Cortez Navarro
- Luther Williams Sandria 
- Ferdinand Bierbaum

In [1]:
import findspark
findspark.init()

#### Creacion de la conexión con el cluster de spark


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("MLSpark-Logistic-Regression") \
    .master("spark://a92682f5ddcc:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext
spark.conf.set("spark.sql.shuffle.partitions", "5")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/25 08:21:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/04/25 08:22:00 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


### Preparación de Datos

In [3]:

import team_name.spark_utils

# Load the data
columns = [("male","integer"),
           ("age","integer"),
           ("education","integer"),
           ("currentSmoker","integer"),
           ("cigsPerDay","integer"),
           ("BPMeds","integer"),
           ("prevalentStroke","integer"),
           ("prevalentHyp","integer"),
           ("diabetes","integer"),
           ("totChol","double"),
           ("sysBP","double"),
           ("diaBP","double"),
           ("BMI","double"),
           ("heartRate","double"),
           ("glucose","double"),
           ("TenYearCHD","integer")]

schema = team_name.spark_utils.SparkUtils.generate_schema(columns)

# Create DataFrame
data = spark \
                .read \
                .schema(schema) \
                .option("header", "true") \
                .csv("/home/jovyan/notebooks/data/framingham.csv")

# If 'NA' is present in the data, replace it with None
data = data.na.replace('NA', None)

# Drop rows with any null values
data = data.na.drop()

### Assemble the features into a single vector column

In [4]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=["age", "education", "currentSmoker", "cigsPerDay",
                                      "BPMeds", "prevalentStroke", "prevalentHyp", "diabetes",
                                      "totChol", "sysBP", "diaBP", "BMI", "heartRate", "glucose"],
                            outputCol="features", handleInvalid="skip")

data_with_features = assembler.transform(data).select("TenYearCHD", "features")


data_with_features.show(15, truncate=False)

25/04/25 08:22:06 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
25/04/25 08:22:22 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/04/25 08:22:37 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/04/25 08:22:52 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/04/25 08:23:07 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
25/04/25 08:23:22 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI

+----------+----------------------------------------------------------------------+
|TenYearCHD|features                                                              |
+----------+----------------------------------------------------------------------+
|0         |(14,[0,1,8,9,10,11,12,13],[39.0,4.0,195.0,106.0,70.0,26.97,80.0,77.0])|
|0         |(14,[0,1,8,9,10,11,12,13],[46.0,2.0,250.0,121.0,81.0,28.73,95.0,76.0])|
|0         |[48.0,1.0,1.0,20.0,0.0,0.0,0.0,0.0,245.0,127.5,80.0,25.34,75.0,70.0]  |
|1         |[61.0,3.0,1.0,30.0,0.0,0.0,1.0,0.0,225.0,150.0,95.0,28.58,65.0,103.0] |
|0         |[46.0,3.0,1.0,23.0,0.0,0.0,0.0,0.0,285.0,130.0,84.0,23.1,85.0,85.0]   |
|0         |[43.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,228.0,180.0,110.0,30.3,77.0,99.0]   |
|1         |(14,[0,1,8,9,10,11,12,13],[63.0,1.0,205.0,138.0,71.0,33.11,60.0,85.0])|
|0         |[45.0,2.0,1.0,20.0,0.0,0.0,0.0,0.0,313.0,100.0,71.0,21.68,79.0,78.0]  |
|0         |[52.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,260.0,141.5,89.0,26.36,76.0,79

### Split the data into training and test sets 80% training data and 20% testing data

In [5]:
train_df, test_df = data_with_features.randomSplit([0.8, 0.2], seed=13)

### Show the whole dataset

In [6]:
print("Original Dataset")
data_with_features.show(truncate=False, n=5)

# Print train dataset
print("train set")
train_df.show(truncate=False, n=5)

Original Dataset


+----------+----------------------------------------------------------------------+
|TenYearCHD|features                                                              |
+----------+----------------------------------------------------------------------+
|0         |(14,[0,1,8,9,10,11,12,13],[39.0,4.0,195.0,106.0,70.0,26.97,80.0,77.0])|
|0         |(14,[0,1,8,9,10,11,12,13],[46.0,2.0,250.0,121.0,81.0,28.73,95.0,76.0])|
|0         |[48.0,1.0,1.0,20.0,0.0,0.0,0.0,0.0,245.0,127.5,80.0,25.34,75.0,70.0]  |
|1         |[61.0,3.0,1.0,30.0,0.0,0.0,1.0,0.0,225.0,150.0,95.0,28.58,65.0,103.0] |
|0         |[46.0,3.0,1.0,23.0,0.0,0.0,0.0,0.0,285.0,130.0,84.0,23.1,85.0,85.0]   |
+----------+----------------------------------------------------------------------+
only showing top 5 rows

train set
+----------+-----------------------------------------------------------------------+
|TenYearCHD|features                                                               |
+----------+---------------------------

### Create a logistic regression model

In [7]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(maxIter=100, regParam=0.01, featuresCol="features", labelCol="TenYearCHD")

# TRAINNING

In [8]:
lr_model = lr.fit(train_df)

# Print coefficients
print("Coefficients: " + str(lr_model.coefficients))

# Display model summary
training_summary = lr_model.summary

25/04/25 08:23:47 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
25/04/25 08:23:47 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


Coefficients: [0.05907035932929528,-0.04815961064690652,0.16561087327641522,0.019049498884948744,0.09755746473120887,0.6877427350809565,0.322764993843067,0.34317101890846613,0.0013385001294625666,0.010282313750778765,-0.0012083272582368541,0.015446583629678125,-0.004334149025324942,0.005879810098869941]


# PREDICTIONS

In [9]:
# Use the trained model to make predictions on the test data
predictions = lr_model.transform(test_df)

# Show predictions
predictions.select("features", "prediction", "probability").show(truncate=False)

# Show real values
predictions.select("features", "TenYearCHD").show(truncate=False)


+-----------------------------------------------------------------------+----------+-----------------------------------------+
|features                                                               |prediction|probability                              |
+-----------------------------------------------------------------------+----------+-----------------------------------------+
|(14,[0,1,8,9,10,11,12,13],[33.0,4.0,165.0,136.0,75.0,24.95,88.0,90.0]) |0.0       |[0.9682176683366963,0.03178233166330369] |
|(14,[0,1,8,9,10,11,12,13],[34.0,3.0,196.0,108.5,68.0,25.67,60.0,82.0]) |0.0       |[0.9694299969145499,0.030570003085450104]|
|(14,[0,1,8,9,10,11,12,13],[35.0,2.0,175.0,121.5,74.5,20.86,92.0,93.0]) |0.0       |[0.9677065521510602,0.03229344784893984] |
|(14,[0,1,8,9,10,11,12,13],[36.0,1.0,200.0,108.0,62.0,20.79,75.0,69.0]) |0.0       |[0.969278682609449,0.03072131739055095]  |
|(14,[0,1,8,9,10,11,12,13],[36.0,1.0,200.0,112.0,76.0,26.03,75.0,62.0]) |0.0       |[0.9673147894804681,0.03268

+-----------------------------------------------------------------------+----------+
|features                                                               |TenYearCHD|
+-----------------------------------------------------------------------+----------+
|(14,[0,1,8,9,10,11,12,13],[33.0,4.0,165.0,136.0,75.0,24.95,88.0,90.0]) |0         |
|(14,[0,1,8,9,10,11,12,13],[34.0,3.0,196.0,108.5,68.0,25.67,60.0,82.0]) |0         |
|(14,[0,1,8,9,10,11,12,13],[35.0,2.0,175.0,121.5,74.5,20.86,92.0,93.0]) |0         |
|(14,[0,1,8,9,10,11,12,13],[36.0,1.0,200.0,108.0,62.0,20.79,75.0,69.0]) |0         |
|(14,[0,1,8,9,10,11,12,13],[36.0,1.0,200.0,112.0,76.0,26.03,75.0,62.0]) |0         |
|(14,[0,1,8,9,10,11,12,13],[36.0,1.0,233.0,128.0,86.5,35.31,83.0,76.0]) |0         |
|(14,[0,1,8,9,10,11,12,13],[36.0,2.0,163.0,111.0,73.0,30.18,70.0,90.0]) |0         |
|(14,[0,1,8,9,10,11,12,13],[36.0,2.0,197.0,115.0,65.0,20.42,68.0,77.0]) |0         |
|(14,[0,1,8,9,10,11,12,13],[36.0,3.0,180.0,111.0,73.0,27.78,71.0,

In [10]:
# Show predictions and real values where prediction is 1
predictions.filter(predictions.prediction == 1).select("features", "TenYearCHD", "prediction").show(truncate=False)

# Show predictions and real values where prediction is 0
predictions.filter(predictions.prediction == 0).select("features", "TenYearCHD", "prediction").show(truncate=False)


+----------------------------------------------------------------------+----------+----------+
|features                                                              |TenYearCHD|prediction|
+----------------------------------------------------------------------+----------+----------+
|[50.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,260.0,190.0,130.0,43.67,85.0,260.0] |0         |1.0       |
|[54.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,326.0,187.0,95.0,29.94,67.0,235.0]  |0         |1.0       |
|[65.0,1.0,1.0,9.0,0.0,0.0,1.0,1.0,271.0,144.5,88.0,32.41,82.0,116.0]  |0         |1.0       |
|[52.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,258.0,177.0,111.0,30.38,80.0,270.0] |1         |1.0       |
|[57.0,1.0,1.0,43.0,0.0,0.0,1.0,0.0,283.0,207.5,118.0,38.61,100.0,83.0]|1         |1.0       |
|[59.0,1.0,1.0,3.0,0.0,0.0,1.0,1.0,230.0,182.0,102.0,25.91,66.0,147.0] |1         |1.0       |
|[62.0,2.0,0.0,0.0,1.0,0.0,1.0,0.0,298.0,248.0,130.0,37.1,96.0,77.0]   |1         |1.0       |
|[64.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,253.0,295.0,135

In [11]:
# Show model accuracy
accuracy = predictions.filter(predictions.TenYearCHD == predictions.prediction).count() / predictions.count()
print(f"Model accuracy: {accuracy:.2%}")

Model accuracy: 86.24%


In [12]:
sc.stop()